# Software Analysis, Testing, and Verification Final Project
### By Adam Traub

### Large-Scale test of PrettyTable

## Part I Getting Started

### A.) Setup

In [3]:
#The following libraries are required
!pip install prettytable
!pip install pytest

### Pytest Quirk

The following code exposed a shortcoming in PyTest that was surprisingly difficult to overcome.

In [ ]:
class TestPrettyTable(object):
    
    #STYLE Expectation
    def __tableStyleHandler(self, style):
        """Tester for table Styles"""
        if style in STYLES: #If it's a good style
            try:
                self.table.set_style(style) #Set the style
            except Exception: #and fail if an exception is raised
                pytest.fail("set_style raised Exception unexpectedly!")

        else: #If it's a bad style
             with pytest.raises(Exception) as context: #expect an exception
                 self.table.set_style(style) #When the bad style is set

Don't see a problem?  It's a suble one.  What happens if a an exception is hit as a result of a mistake in the testing code inside the else statement?  if that happens, Pytest will count that as the test passing because it's only looking for a raised exception.<br>
<br>
The first possible fix is an obvious one - use a more specific exception than the generic "Exception."  This would be a great solution except that the method being tested uses the generic Exception class `raise Exception("Invalid pre-set style!")`<br>
<br>
My workaround was a simple one.  We want to confirm that the error we hit has the `Invalid pre-set style` string associated with it.  The following code was my attempt to circumvent this shortcoming.

In [4]:
class TestPrettyTable(object):

    #STYLE Expectation
    def __tableStyleHandler(self, style):
        """Tester for table Styles"""
        if style in STYLES: #If it's a good style
            try:
                self.table.set_style(style) #Set the style
            except Exception: #and fail if an exception is raised
                pytest.fail("set_style raised Exception unexpectedly!")

        else: #If it's a bad style
            try:
                self.table.set_style(style) #Try to set the bad style
                pytest.fail("Exception failed to hit") #Fail if no exception is hit
            except Exception as e:
                #Assert it's the correct exception
                assert(str(e) == "Invalid pre-set style!") 

While the new code is arguably more complex, it does narrow our test to the precise circumstances of this error and prevents false negatives.